In [1]:
%load_ext autoreload
%autoreload 2


In [3]:
# Train the ML model
from utils import sample_files, encoder_files_to_tensors
from utils import plot_loss, load_encoder_data
from models import Encoder

import time
import glob
import tensorflow as tf
from tensorflow import keras
import yaml
import os
import numpy as np
from datetime import datetime
import argparse
import matplotlib.pyplot as plt
import matplotlib as mpl
num_Turns_Case = 50+1


2022-10-24 09:14:57.299477: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-24 09:14:57.299501: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
# Initialize parameters
# data_dir = '/eos/kiliakis/tomo_data/datasets'
data_dir = './tomo_data/datasets'

timestamp = datetime.now().strftime("%Y_%m_%d_%H-%M-%S")
print('Using timestamp: ', timestamp)



Using timestamp:  2022_10_24_09-14-59


In [5]:
# Data specific
IMG_OUTPUT_SIZE = 128
# BUFFER_SIZE = 256
BATCH_SIZE = 128  # 8
latent_dim = 7  # 6 + the new VrfSPS
# additional_latent_dim = 1

# Train specific
train_cfg = {
    'epochs': 10,
    'dense_layers': [64, latent_dim],
    'filters': [4, 8],
    'cropping': [0, 0],
    'kernel_size': 7,
    'strides': [2, 2],
    'activation': 'relu',
    'pooling': None,
    'pooling_size': [0, 0],
    'pooling_strides': [1, 1],
    'pooling_padding': 'valid',
    'dropout': 0.2,
    'loss': 'mse',
    'lr': 1e-3,
    'dataset%': 0.1,
    'normalization': 'minmax',
    'loss_weights': [1, 1, 1, 1, 1, 1, 1]

}



In [6]:
# Initialize directories
trial_dir = os.path.join('./trials/', timestamp)
weights_dir = os.path.join(trial_dir, 'weights')
plots_dir = os.path.join(trial_dir, 'plots')

# Initialize train/ test / validation paths
ML_dir = os.path.join(data_dir, 'ML_data')
TRAINING_PATH = os.path.join(ML_dir, 'TRAINING')
# VALIDATION_PATH = os.path.join(ML_dir, 'VALIDATION')
assert os.path.exists(TRAINING_PATH)
# assert os.path.exists(VALIDATION_PATH)

# create the directory to store the results
os.makedirs(trial_dir, exist_ok=True)
os.makedirs(weights_dir, exist_ok=False)
os.makedirs(plots_dir, exist_ok=False)

# Initialize GPU
gpus = tf.config.experimental.list_physical_devices('GPU')
device_to_use = 0

if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        tf.config.experimental.set_memory_growth(gpus[device_to_use], True)
        tf.config.experimental.set_virtual_device_configuration(
            gpus[device_to_use],
            [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=12*1024)])
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(
            logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)
else:
    print('No GPU available, using the CPU')


No GPU available, using the CPU


2022-10-24 09:15:33.192985: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-24 09:15:33.193006: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-24 09:15:33.193020: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (kiliakis-ubuntu): /proc/driver/nvidia/version does not exist


In [7]:
start_t = time.time()
# Create the datasets
# 1. Randomly select the training data
file_names = sample_files(TRAINING_PATH, train_cfg['dataset%'], keep_every=num_Turns_Case)
print('Number of Training files: ', len(file_names))

x_train, y_train = encoder_files_to_tensors(
    file_names, normalization=train_cfg['normalization'])
'''
# 2. Convert to tensor dataset
train_dataset = tf.data.Dataset.from_tensor_slices(file_names)

# 3. Then map function to dataset
# this returns pairs of tensors with shape (128, 128, 1) and (7,)
train_dataset = train_dataset.map(lambda x: tf.py_function(
    load_encoder_data,
    [x, train_cfg['normalization'], True],
    [tf.float32, tf.float32]))

# 4. Ignore errors in case they appear
train_dataset = train_dataset.apply(
    tf.data.experimental.ignore_errors())

# 6. Divide dataset in batces
train_dataset = train_dataset.batch(BATCH_SIZE)
'''

# Repeat for validation data
# file_names = sample_files(
#     VALIDATION_PATH, train_cfg['dataset%'], keep_every=num_Turns_Case)
# print('Number of Validation files: ', len(file_names))

# x_valid, y_valid = encoder_files_to_tensors(
#     file_names, normalization=train_cfg['normalization'])

'''
# convert to dataset
valid_dataset = tf.data.Dataset.from_tensor_slices(file_names)
# Then map function to dataset
# this returns pairs of tensors with shape (128, 128, 1) and (7,)
valid_dataset = valid_dataset.map(lambda x: tf.py_function(
    load_encoder_data,
    [x, train_cfg['normalization'], True],
    [tf.float32, tf.float32]))
# Ignore errors
valid_dataset = valid_dataset.apply(
    tf.data.experimental.ignore_errors())

# cache the dataset
# valid_dataset = valid_dataset.cache(
#     os.path.join(cache_dir, 'valid_cache'))
# batch the dataset
valid_dataset = valid_dataset.batch(BATCH_SIZE)
'''

end_t = time.time()
print(
    f'\n---- Input files have been read, elapsed: {end_t - start_t} ----\n')


Number of Training files:  195


2022-10-24 09:15:40.392400: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.



---- Input files have been read, elapsed: 0.2423388957977295 ----



In [ ]:
# print(tf.reduce_mean(y_train, 0))
# print(tf.math.reduce_std(y_train, 0))

In [ ]:
%matplotlib inline
# plot some of the outputs

nrows = 3
# Get nrows * nrows random images
sample = np.random.choice(np.arange(len(x_train)),
                          size=nrows * nrows, replace=False)

samples_X = tf.gather(x_train, sample)
samples_y = tf.gather(y_train, sample)

# Create 3x3 grid of figures
fig, axes = plt.subplots(ncols=nrows, nrows=nrows, figsize=(12, 12))
axes = np.ravel(axes)
for i in range(len(axes)):
    ax = axes[i]
    ax.set_xticks([])
    ax.set_yticks([])
    # show the image
    ax.imshow(samples_X[i], cmap='jet')
    # Set the label
    title = ','.join([f'{num:.1f}' for num in samples_y[i]])
    ax.set_title(f'{title}')


In [8]:
# Model instantiation

input_shape = (IMG_OUTPUT_SIZE, IMG_OUTPUT_SIZE, 1)

encoder = Encoder(input_shape=input_shape, **train_cfg)

print(encoder.model.summary())



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Crop (Cropping2D)           (None, 128, 128, 1)       0         
                                                                 
 CNN_1 (Conv2D)              (None, 61, 61, 4)         200       
                                                                 
 CNN_2 (Conv2D)              (None, 28, 28, 8)         1576      
                                                                 
 Flatten (Flatten)           (None, 6272)              0         
                                                                 
 Dense_1 (Dense)             (None, 64)                401472    
                                                                 
 Dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 Output (Dense)              (None, 7)                 4

In [9]:
# Train the encoder

# callbacks, save the best model, and early stop if no improvement in val_loss
stop_early = keras.callbacks.EarlyStopping(monitor='val_loss',
                                           patience=5, restore_best_weights=True)
save_best = keras.callbacks.ModelCheckpoint(filepath=os.path.join(weights_dir, 'encoder.h5'),
                                            monitor='val_loss', save_best_only=True)


In [12]:
start_time = time.time()
# history = encoder.model.fit(
#     train_dataset, epochs=train_cfg['epochs'],
#     validation_data=valid_dataset,
#     callbacks=[stop_early, save_best])
history = encoder.model.fit(
    x_train, y_train, epochs=train_cfg['epochs'],
    # validation_data=(x_valid, y_valid), 
    batch_size=BATCH_SIZE,
    callbacks=[stop_early, save_best])

total_time = time.time() - start_time


Epoch 1/10
2/2 [==============================] - 0s 67ms/step - loss: 0.1025
Epoch 2/10
2/2 [==============================] - 0s 66ms/step - loss: 0.1041
Epoch 3/10
2/2 [==============================] - 0s 56ms/step - loss: 0.1075
Epoch 4/10
2/2 [==============================] - 0s 53ms/step - loss: 0.0909
Epoch 5/10
2/2 [==============================] - 0s 64ms/step - loss: 0.0936
Epoch 6/10
2/2 [==============================] - 0s 63ms/step - loss: 0.0953
Epoch 7/10
2/2 [==============================] - 0s 61ms/step - loss: 0.0817
Epoch 8/10
2/2 [==============================] - 0s 63ms/step - loss: 0.0821
Epoch 9/10
2/2 [==============================] - 0s 53ms/step - loss: 0.0782
Epoch 10/10
2/2 [==============================] - 0s 50ms/step - loss: 0.0791


In [13]:
print(history.history)

{'loss': [0.10253168642520905, 0.10409383475780487, 0.1075415089726448, 0.09088212251663208, 0.09360166639089584, 0.09526021778583527, 0.08166756480932236, 0.0820583626627922, 0.07816998660564423, 0.07914452999830246]}


In [ ]:
from sklearn.metrics import mean_absolute_error

from functools import partial


def my_mse(y_true, y_pred):
    return tf.reduce_mean(tf.math.abs(y_pred - y_true), axis=0)


y_true = np.array([[0., 0.], [1., 1.]])
y_pred = np.array([[1., 0.5], [2., 1.5]])
loss = keras.losses.mean_absolute_error(y_true, y_pred)
mae2 = MyMeanAbsoluteError(reduction=keras.losses.Reduction.NONE)
mae = keras.losses.MeanAbsoluteError(reduction=keras.losses.Reduction.NONE)
lossv2 = mean_absolute_error(y_true, y_pred, multioutput='raw_values')
# lossv3 = my_mse(1, y_true, y_pred)
# print(loss)
# print(mae(y_true, y_pred).numpy())
print(lossv2)
# print(mae2(y_true, y_pred).numpy())
# lossv4 = my_mse(y_true, y_pred)

print(my_mse(y_true, y_pred))
# print(my_mse(1, y_true, y_pred))


In [ ]:
# Plot training and validation loss
train_loss_l = np.array(history.history['loss'])
valid_loss_l = np.array(history.history['val_loss'])

plot_loss({'Training': train_loss_l, 'Validation': valid_loss_l},
          title='Encoder Train/Validation Loss',
          figname=os.path.join(plots_dir, 'encoder_train_valid_loss.png'))


In [ ]:
from sklearn.metrics import mean_squared_error

# get predictions
y_pred = encoder.model.predict(x_valid, verbose=False)
y_valid = np.array(y_valid)

# Calculate error per variable
mses = mean_squared_error(y_valid, y_pred, multioutput='raw_values')

var_names = ['phase_error', 'energy_error',
             'bunch_length', 'intensity', 'Volt_rf', 'mu', 'Vrf_SPS']
# report
print('Variable\tMSE')
for name, mse in zip(var_names, mses):
    print(f'{name}:\t{mse:.4f}')


In [ ]:
# save file with experiment configuration
config_dict = {}
config_dict['encoder'] = train_cfg.copy()
config_dict['encoder'].update({
    'min_train_loss': float(np.min(train_loss_l)),
    'min_valid_loss': float(np.min(valid_loss_l)),
    'total_train_time': total_time,
    'used_gpus': len(gpus)
})

# save config_dict
with open(os.path.join(trial_dir, 'encoder-summary.yml'), 'w') as configfile:
    yaml.dump(config_dict, configfile, default_flow_style=False)
